#Traitement et Préparation de Données Audio pour un Dataset

##### Ce code est conçu pour lire des fichiers audio au format WAV, les convertir en un format manipulable par NumPy, puis les intégrer dans un dataset au format compatible avec les bibliothèques de machine learning comme Hugging Face datasets. Le processus comprend également un nettoyage du texte associé aux fichiers audio.

### Importation des Bibliothèques

#### Le code commence par l'importation des bibliothèques nécessaires :

In [ ]:
!pip install datasets

In [ ]:
import wave
import numpy as np
from datasets import Dataset, Features, Value, Audio, load_dataset
import pandas as pd

##### Bibliothèques utilisées :



*   wave : Utilisé pour manipuler les fichiers audio au format WAV.
* numpy : Manipule les données sous forme de tableaux multidimensionnels.
* datasets : Une bibliothèque de Hugging Face utilisée pour créer et manipuler des datasets.
* pandas : Utilisé pour la gestion et la manipulation des fichiers CSV contenant des métadonnées.



### Lecture d'un Fichier WAV

###### La fonction suivante lit un fichier audio au format WAV et renvoie ses données sous forme de tableau NumPy ainsi que la fréquence d'échantillonnage :

In [ ]:
def read_wav_file(file_path):
    """Reads a WAV file and returns the audio data as a NumPy array.
    Args:
        file_path: Path to the WAV file.
    Returns:
        A tuple containing:
            - The audio data as a NumPy array.
            - The sample rate of the audio.
    """
    with wave.open(file_path, 'rb') as wf:
        num_channels = wf.getnchannels()
        sample_width = wf.getsampwidth()
        frame_rate = wf.getframerate()
        num_frames = wf.getnframes()
        audio_data = wf.readframes(num_frames)
        audio_data = np.frombuffer(audio_data, dtype=np.int16)
        if num_channels == 2:
            audio_data = audio_data.reshape(-1, 2)
    return audio_data, frame_rate

#### Explication :






*   wave.open() ouvre le fichier audio.

*   getnchannels(), getsampwidth(), getframerate(), et getnframes() récupèrent des informations importantes
 comme le nombre de canaux, la largeur des échantillons, la fréquence d'échantillonnage et le nombre d'images.
*   readframes() lit les données audio brutes.


*   np.frombuffer() convertit ces données brutes en un tableau NumPy.

*   Si le fichier audio a deux canaux (stéréo), il est réorganisé en deux colonnes.





### Chargement du Fichier CSV de Métadonnées

##### Le fichier CSV est chargé avec pandas et contient les noms des fichiers audio ainsi que le texte associé :

In [ ]:
data = pd.read_csv('/folder_path/metadata.csv', sep='|', header=None, on_bad_lines='skip')

###### Ici, metadata.csv contient des informations comme le nom des fichiers audio et leur transcription associée. Le séparateur utilisé est le caractère |, et la gestion des lignes incorrectes est configurée pour les sauter

### Nettoyage et Préparation des Données

#### La fonction suivante est utilisée pour lire les fichiers audio et nettoyer le texte correspondant :

In [ ]:
def cleaner(el, text):
    audio_array, sampling_rate = read_wav_file(f'/folder_path/wavs/{el}')
    text_cleaned = text.replace(';', "")
    return {
        "text": text_cleaned,
        "gender": "female",
        "audio": {"array": audio_array, "sampling_rate": sampling_rate}
        }


#### Explication :



*    La fonction cleaner lit les données audio en appelant la fonction read_wav_file() pour chaque fichier mentionné dans le fichier CSV.
*   Elle nettoie également le texte associé en supprimant certains caractères indésirables (comme les points-virgules).

*   Les données sont ensuite renvoyées sous la forme d'un dictionnaire contenant l'audio (tableau et taux
 d'échantillonnage), le texte nettoyé, et un attribut de genre (ici fixé à "female")



### Création d'un Dictionnaire

#####  Le code suivant applique la fonction cleaner à chaque élément et transcription du fichier CSV pour créer une liste de dictionnaires :

In [ ]:
dic = [cleaner(el, text) for el, text in zip(data[0], data[1])]

### Création et Formatage du Dataset

#### Les données sont ensuite transformées en un dataset Hugging Face :

In [ ]:
dataset = Dataset.from_list(dic)

#### Ensuite, les features (caractéristiques) du dataset sont définies, précisant que la colonne audio contient des données audio, et la colonne text contient des chaînes de caractères.


In [ ]:
features = Features({
    'audio': Audio(sampling_rate=16000),
    'text': Value('string'),
})

### Casting du Dataset

#### Finalement, les features sont appliquées au dataset en utilisant la méthode cast() :

In [ ]:
dataset = dataset.cast(features)

#### Cela assure que les données audio sont correctement formatées avec une fréquence d'échantillonnage de 16 000 Hz et que le texte est reconnu comme des chaînes de caractères.



---



###  Conclusion

####  Ce code permet de transformer des fichiers audio WAV et leurs métadonnées associées en un dataset structuré, prêt à être utilisé dans des tâches de traitement automatique du langage ou d'apprentissage machine, en particulier pour des modèles de Text-to-Speech (TTS) ou de reconnaissance vocale.